### Power simulation

Test of comparative statics

In [1]:
from itertools import product

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, multivariate_normal

In [2]:
def compute_test_result(n_matching_grp_per_treatment, 
                        n_obs_per_matching_grp,
                        small_coal_worth, 
                        unit, 
                        err_type, 
                        err_scale, 
                        ind_set, 
                        test,
                        use_treatment_dummies, 
                        cov_type):
    """
    Run a statistical test on data generated by assumed DGP

    Parameters: 
    - experiment set-up
        * n_matching_grp_per_treatment # number of matching groups per treatment/session
        * n_obs_per_matching_grp # number of observations per matching group
        * small_coal_worth # list of treatment values
    - DGP:
        * err_type = "uniform" # choose: "uniform", "norm_corr"
        * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
        * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
    - specification:
        * unit # choose: "group", "matching_group"
        * use_treatment_dummies # choose: 0,1
        * test # choose: "reg", "mann_whitney" (one-sided)
        * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3" (if using group as a unit)
    
    Returns:
    - p value(s) 
    """
    # Generate independent variables
    
    n_treat = len(small_coal_worth)
    n_obs = n_obs_per_matching_grp * n_matching_grp_per_treatment * n_treat # total number of group observations
    
    Xval = np.array(sum([n_obs_per_matching_grp * n_matching_grp_per_treatment * [val] for val in small_coal_worth], [])) 
    X = pd.get_dummies(Xval, drop_first= True).values.astype(int) if use_treatment_dummies else Xval 
    X = sm.add_constant(X)

    if unit == "matching_group" or (unit == "group" and cov_type == "cluster"): 
        clusters = np.array(sum([n_obs_per_matching_grp * [cluster] for cluster in range(0, n_matching_grp_per_treatment * n_treat)], [])) 
        X = np.column_stack((X, clusters))

    # Assumed DGP 

    if err_type == "uniform":
        e = np.random.random(n_obs) * 2 * err_scale - err_scale
    elif err_type == "norm_corr": # generate correlated errors within each matching group
        cov = err_scale * np.random.random(size=(n_obs_per_matching_grp,n_obs_per_matching_grp))
        cov = cov + cov.T # make it symmetric 
        cov = np.dot(cov, cov.T) # make it positive semidefinite
        e = multivariate_normal.rvs(mean = np.zeros(n_obs_per_matching_grp), cov = cov, size = n_matching_grp_per_treatment * n_treat).flatten()
    e = e.round() # as subjects can only choose integer values

    es = np.ones(n_obs) * (100//3)  
    sh = (100 + Xval)//3  
    no_coord = np.zeros(n_obs)
    ind = {0: es, 1: sh, 2: no_coord} 

    rand = [np.random.choice(ind_set) for j in range(n_obs)] 
    y = np.array([ind[rand[i]][i] for i in range(n_obs)]) + np.array([0 if rand[i]==2 else e[i] for i in range(n_obs)])
    y = np.maximum(y,0) 
    y = np.minimum(y,100)

    if unit == "matching_group":
        y = np.array([np.mean(y[X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])
        X = np.array([np.mean(X[:,0:-1][X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])

    # Run statistical test (only on outcomes with successful coordination)

    y_reg = y[y>0] 
    X_reg = X[y>0]

    if test == "reg":
        if unit == "matching_group":
            results = sm.regression.linear_model.OLS(y_reg, X_reg).fit() 
        elif unit == "group":
            if cov_type == "cluster":
                results = sm.regression.linear_model.OLS(y_reg, X_reg[:, :-1]).fit(cov_type = cov_type, cov_kwds= {"groups": X_reg[:, -1]}) # clustered standard errors
            else:
                results = sm.regression.linear_model.OLS(y_reg, X_reg).fit(cov_type = cov_type) # robust standard errors

        return (results.pvalues[1], results.pvalues[2]) if use_treatment_dummies else results.pvalues[1]

    elif test == "mann_whitney": 
        # note that 'power_coeff1' and 'power_coeff2' take on a different meaning here than for regression
        results_1 = mannwhitneyu(y_reg[X_reg[:,1] == small_coal_worth[0]], y_reg[X_reg[:,1] == small_coal_worth[1]], alternative = "less", method = "auto")
        results_2 = mannwhitneyu(y_reg[X_reg[:,1] == small_coal_worth[0]], y_reg[X_reg[:,1] == small_coal_worth[2]], alternative = "less", method = "auto")
        
        return (results_1.pvalue, results_2.pvalue)

In [3]:
def power_simulation(num_runs, 
                     sig_level, 
                     n_matching_grp_per_treatment, 
                     n_obs_per_matching_grp, 
                     small_coal_worth, 
                     unit, 
                     err_type, 
                     err_scale,
                     ind_set, 
                     test,
                     use_treatment_dummies, 
                     cov_type):
    """
        Run a power simulation for our bargaining experiment 
    
        Parameters: 
        - simulation:
            * num_runs 
            * sig_level 
        - experiment set-up
            * n_matching_grp_per_treatment # number of matching groups per treatment/session
            * n_obs_per_matching_grp # number of observations per matching group
            * small_coal_worth # list of treatment values
        - DGP:
            * err_type = "uniform" # choose: "uniform", "norm_corr"
            * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
            * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
        - specification:
            * unit # choose: "group", "matching_group"
            * test # choose: "reg", "mann_whitney" (one-sided)
            * use_treatment_dummies # choose: 0,1
            * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3"
        
        Returns: 
        - Dataframe with p value(s) for each simulation run.
    """
    power_sim_results = pd.DataFrame({'p_value_1': np.zeros(num_runs), 'p_value_2': np.zeros(num_runs)}) if (use_treatment_dummies or test == "mann_whitney") else pd.DataFrame({'p_value_1': np.zeros(num_runs)}) 

    for run in range(num_runs):
        power_sim_results.loc[run, :] = compute_test_result(n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, test, use_treatment_dummies, cov_type)

    power_sim_summary = dict([(key,value) for key, value in locals().items() if key not in ["power_sim_results", "run"]])

    p_reject_1 = np.mean(power_sim_results['p_value_1'] < sig_level)
    if use_treatment_dummies or test == "mann_whitney": 
        p_reject_2 = np.mean(power_sim_results['p_value_2'] < sig_level)
        
    power_sim_summary["power_coeff1"] = p_reject_1 
    power_sim_summary["power_coeff2"] = p_reject_2 if (use_treatment_dummies or test == "mann_whitney") else np.nan

    return power_sim_summary


In [4]:
# run power simulation over a set of plausible parameters

# parameters: simulation, DGP
num_runs = [5000]
sig_level = [0.05]
n_matching_grp_per_treatment = [6]
n_obs_per_matching_grp = [10]
small_coal_worth = [[10,30,90]]
err_type = ["uniform", "norm_corr"] 
err_scale =  [1,2,3]
ind_set = [[0,0,0,1,1,1,2], [0,1,1,1,1,2]]

# parameters: test specification 
# (use subset of a grid, as not all combinations make sense)
params = {  
            0: {"test": "reg", "unit": "group", "use_treatment_dummies": 1, "cov_type": "cluster"},
            1: {"test": "reg", "unit": "group", "use_treatment_dummies": 1, "cov_type": "HC3"},
            2: {"test": "reg", "unit": "group", "use_treatment_dummies": 0, "cov_type": "cluster"},
            3: {"test": "reg", "unit": "group", "use_treatment_dummies": 0, "cov_type": "HC3"},
            4: {"test": "mann_whitney", "unit": "group", "use_treatment_dummies": 0, "cov_type": np.nan},
            5: {"test": "reg", "unit": "matching_group", "use_treatment_dummies": 1, "cov_type": np.nan},
            6: {"test": "mann_whitney", "unit": "matching_group", "use_treatment_dummies": 0, "cov_type": np.nan},
            7: {"test": "reg", "unit": "matching_group", "use_treatment_dummies": 0, "cov_type": np.nan}
         }

# simulation
res = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, err_type, err_scale, ind_set, params))
for comb in combinations: 
    #print(comb)
    
    ps = power_simulation(   
            num_runs = comb[0], 
            sig_level = comb[1], 
            n_matching_grp_per_treatment = comb[2], 
            n_obs_per_matching_grp = comb[3], 
            small_coal_worth = comb[4], 
            unit = params[comb[8]]["unit"], 
            err_type = comb[5], 
            err_scale = comb[6],
            ind_set = comb[7], 
            test = params[comb[8]]["test"],
            use_treatment_dummies = params[comb[8]]["use_treatment_dummies"], 
            cov_type = params[comb[8]]["cov_type"]
        )
    r = pd.DataFrame.from_dict(ps, orient = "index").transpose()
    res = pd.concat([res, r], axis = 0)
    

Basic analysis/comparison of results
* cov_type: cluster vs HC3
* unit: group vs matching group
* test: parametric vs nonparametric

In [5]:
# make them hashable
res["small_coal_worth"] = res["small_coal_worth"].apply(lambda x: tuple(x))
res["ind_set"] = res["ind_set"].apply(lambda x: tuple(x))

In [6]:
res_pivot = res.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["test", "unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_pivot

power_coeff1                  \
test                                               reg                   
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                1       1       0   
err_type  err_scale ind_set                                              
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)       0.4892  0.7868  0.9964   
                    (0, 1, 1, 1, 1, 2)          0.8328  0.9724     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.2148   0.621  0.8916   
                    (0, 1, 1, 1, 1, 2)          0.3826  0.7794   0.999   
          3         (0, 0, 0, 1, 1, 1, 2)        0.147  0.5594  0.6798   
                    (0, 1, 1, 1, 1, 2)           0.238  0.6524   0.964   
uniform   1         (0, 0, 0, 1, 1, 1, 2)       0.9952   0.998     1.0   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)        0.995  0.9954     1.0   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)        0.992  0.9874  0.9998   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   

                                                                               \
test                                              mann_whitney            reg   
unit                                                     group matching_group   
cov_type                                      HC3          NaN            NaN   
use_treatment_dummies                           0            0              1   
err_type  err_scale ind_set                                                     
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)  0.9992       0.7728         0.0958   
                    (0, 1, 1, 1, 1, 2)        1.0        0.988         0.1908   
          2         (0, 0, 0, 1, 1, 1, 2)  0.9756       0.5944         0.0838   
                    (0, 1, 1, 1, 1, 2)     0.9998       0.8082          0.135   
          3         (0, 0, 0, 1, 1, 1, 2)  0.9044       0.4892         0.0738   
                    (0, 1, 1, 1, 1, 2)     0.9962         0.66         0.1022   
uniform   1         (0, 0, 0, 1, 1, 1, 2)     1.0        0.712         0.1094   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.2204   
          2         (0, 0, 0, 1, 1, 1, 2)     1.0       0.7746         0.1162   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.2354   
          3         (0, 0, 0, 1, 1, 1, 2)     1.0       0.8692         0.1128   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0          0.232   

                                                                         \
test                                        mann_whitney            reg   
unit                                      matching_group matching_group   
cov_type                                             NaN            NaN   
use_treatment_dummies                                  0              0   
err_type  err_scale ind_set                                               
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)         0.2292          0.901   
                    (0, 1, 1, 1, 1, 2)            0.3722         0.9914   
          2         (0, 0, 0, 1, 1, 1, 2)         0.1466         0.7068   
                    (0, 1, 1, 1, 1, 2)            0.2554          0.958   
          3         (0, 0, 0, 1, 1, 1, 2)         0.1152         0.5196   
                    (0, 1, 1, 1, 1, 2)            0.1928         0.8592   
uniform   1         (0, 0, 0, 1, 1, 1, 2)         0.2852         0.9372   
                    (0, 1, 1, 1, 1, 2)            0.4848         0.9972   
          2         (0, 0, 0, 1, 1, 1, 2)          0.295         0.9416   
                    (0, 1, 1, 1, 1, 2)            0.4828          0.998   
          3         (0, 0, 0, 1, 1, 1, 2)         

In [7]:
num_cases = len(res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 1)] ) # num of DGPs

In [8]:
# comparison of cov_type: cluster vs HC3 (when unit == group, test = reg) == > more power with HC3
print( (res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 0)] <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff2', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff1','reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)


1.0
1.0
0.9166666666666666


In [9]:
# comparison of unit: matching group vs group = > more power with group
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff2', 'reg', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff2', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1', 'mann_whitney', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'mann_whitney', 'group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'mann_whitney', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff2', 'mann_whitney', 'group', np.nan, 0)]).astype(int).sum()/num_cases)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6666666666666666


In [10]:
# comparison of test: regression (parametric) vs Mann-Whitney (nonparametric)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff1', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'group', 'HC3', 1)] <= res_pivot[('power_coeff1', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases) 

print( (res_pivot[('power_coeff2','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'group', 'HC3', 1)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 0)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'group', 'HC3', 0)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'matching_group', np.nan, 0)] <= res_pivot[('power_coeff2', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'matching_group', np.nan, 1)] <= res_pivot[('power_coeff1', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'matching_group', np.nan , 1)] <= res_pivot[('power_coeff2', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)

0.75
0.5
0.6666666666666666
0.5
0.6666666666666666
0.5
0.16666666666666666
1.0
0.4166666666666667


In [11]:
res_pivot

power_coeff1                  \
test                                               reg                   
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                1       1       0   
err_type  err_scale ind_set                                              
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)       0.4892  0.7868  0.9964   
                    (0, 1, 1, 1, 1, 2)          0.8328  0.9724     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.2148   0.621  0.8916   
                    (0, 1, 1, 1, 1, 2)          0.3826  0.7794   0.999   
          3         (0, 0, 0, 1, 1, 1, 2)        0.147  0.5594  0.6798   
                    (0, 1, 1, 1, 1, 2)           0.238  0.6524   0.964   
uniform   1         (0, 0, 0, 1, 1, 1, 2)       0.9952   0.998     1.0   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)        0.995  0.9954     1.0   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)        0.992  0.9874  0.9998   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   

                                                                               \
test                                              mann_whitney            reg   
unit                                                     group matching_group   
cov_type                                      HC3          NaN            NaN   
use_treatment_dummies                           0            0              1   
err_type  err_scale ind_set                                                     
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)  0.9992       0.7728         0.0958   
                    (0, 1, 1, 1, 1, 2)        1.0        0.988         0.1908   
          2         (0, 0, 0, 1, 1, 1, 2)  0.9756       0.5944         0.0838   
                    (0, 1, 1, 1, 1, 2)     0.9998       0.8082          0.135   
          3         (0, 0, 0, 1, 1, 1, 2)  0.9044       0.4892         0.0738   
                    (0, 1, 1, 1, 1, 2)     0.9962         0.66         0.1022   
uniform   1         (0, 0, 0, 1, 1, 1, 2)     1.0        0.712         0.1094   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.2204   
          2         (0, 0, 0, 1, 1, 1, 2)     1.0       0.7746         0.1162   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.2354   
          3         (0, 0, 0, 1, 1, 1, 2)     1.0       0.8692         0.1128   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0          0.232   

                                                                         \
test                                        mann_whitney            reg   
unit                                      matching_group matching_group   
cov_type                                             NaN            NaN   
use_treatment_dummies                                  0              0   
err_type  err_scale ind_set                                               
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)         0.2292          0.901   
                    (0, 1, 1, 1, 1, 2)            0.3722         0.9914   
          2         (0, 0, 0, 1, 1, 1, 2)         0.1466         0.7068   
                    (0, 1, 1, 1, 1, 2)            0.2554          0.958   
          3         (0, 0, 0, 1, 1, 1, 2)         0.1152         0.5196   
                    (0, 1, 1, 1, 1, 2)            0.1928         0.8592   
uniform   1         (0, 0, 0, 1, 1, 1, 2)         0.2852         0.9372   
                    (0, 1, 1, 1, 1, 2)            0.4848         0.9972   
          2         (0, 0, 0, 1, 1, 1, 2)          0.295         0.9416   
                    (0, 1, 1, 1, 1, 2)            0.4828          0.998   
          3         (0, 0, 0, 1, 1, 1, 2)         

In [12]:
# share of DGPs in grid with power >= 0.8, 0.9 
print(((res_pivot >= 0.8).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())
print(((res_pivot >= 0.9).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())

0.6794871794871795
0.5961538461538461


Test rejection rate when null is true/ type I error

In [13]:
# test rejection rate when null is true
ind_set = [[0]]

# simulation
res_null = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, err_type, err_scale, ind_set, params))
for comb in combinations: 
    #print(comb)
    
    ps = power_simulation(   
            num_runs = comb[0], 
            sig_level = comb[1], 
            n_matching_grp_per_treatment = comb[2], 
            n_obs_per_matching_grp = comb[3], 
            small_coal_worth = comb[4], 
            unit = params[comb[8]]["unit"], 
            err_type = comb[5], 
            err_scale = comb[6],
            ind_set = comb[7], 
            test = params[comb[8]]["test"],
            use_treatment_dummies = params[comb[8]]["use_treatment_dummies"], 
            cov_type = params[comb[8]]["cov_type"]
        )
    r = pd.DataFrame.from_dict(ps, orient = "index").transpose()
    res_null = pd.concat([res_null, r], axis = 0)

In [14]:
res_null["small_coal_worth"] = res_null["small_coal_worth"].apply(lambda x: tuple(x))
res_null["ind_set"] = res_null["ind_set"].apply(lambda x: tuple(x))

res_null_pivot = res_null.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["test", "unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_null_pivot

power_coeff1                                       \
test                                 reg                         mann_whitney   
unit                               group                                group   
cov_type                         cluster     HC3 cluster     HC3          NaN   
use_treatment_dummies                  1       1       0       0            0   
err_type  err_scale ind_set                                                     
norm_corr 1         (0,)           0.096   0.544  0.0814  0.5354       0.2938   
          2         (0,)          0.0958  0.5474  0.0828  0.5434       0.3054   
          3         (0,)           0.095   0.534  0.0812  0.5332       0.2888   
uniform   1         (0,)          0.0936  0.0546  0.0892   0.055       0.0524   
          2         (0,)          0.0862   0.049  0.0924  0.0518       0.0472   
          3         (0,)           0.106   0.053  0.0816  0.0486       0.0454   

                                                                          \
test                                   reg   mann_whitney            reg   
unit                        matching_group matching_group matching_group   
cov_type                               NaN            NaN            NaN   
use_treatment_dummies                    1              0              0   
err_type  err_scale ind_set                                                
norm_corr 1         (0,)            0.0482         0.0466          0.047   
          2         (0,)             0.055         0.0478         0.0462   
          3         (0,)             0.055         0.0442         0.0522   
uniform   1         (0,)            0.0438         0.0424         0.0478   
          2         (0,)            0.0502         0.0398         0.0524   
          3         (0,)            0.0454         0.0408         0.0468   

                            power_coeff2                                    \
test                                 reg                      mann_whitney   
unit                               group                             group   
cov_type                         cluster     HC3 cluster  HC3          NaN   
use_treatment_dummies                  1       1       0    0            0   
err_type  err_scale ind_set                                                  
norm_corr 1         (0,)          0.0936   0.542     NaN  NaN       0.2988   
          2         (0,)          0.0884  0.5462     NaN  NaN       0.2972   
          3         (0,)          0.0968  0.5378     NaN  NaN       0.2954   
uniform   1         (0,)          0.0948  0.0522     NaN  NaN       0.0524   
          2         (0,)          0.0928  0.0526     NaN  NaN         0.05   
          3         (0,)             0.1  0.0516     NaN  NaN       0.0462   

                                                                          
test                                   reg   mann_whitney            reg  
unit                        matching_group matching_group matching_group  
cov_type                               NaN            NaN            NaN  
use_treatment_dummies                    1              0              0  
err_type  err_scale ind_set                                               
norm_corr 1         (0,)            0.0498         0.0466            NaN  
          2         (0,)             0.054         0.0478            NaN  
          3         (0,)            0.0488          0.042            NaN  
uniform   1         (0,)            0.0508         0.0398            NaN  
          2         (0,)            0.0438         0.0434            NaN  
          3         (0,)            0.0474         0.0428            NaN